# Problem

Reverse engineer how a 1L attn-only transformer solves the [majority element](https://leetcode.com/problems/majority-element/) problem.

# Setup
(No need to read)

In [1]:
!git clone https://github.com/ckkissane/mech-interp-practice.git

Cloning into 'mech-interp-practice'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 319 (delta 79), reused 32 (delta 14), pack-reused 181
Receiving objects: 100% (319/319), 38.61 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-q3by3v39
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-q3by3v39
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 54d548de4995a1ecc5b01b9c03aceaf0966c0eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
try:
    %pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python
except:
    import os; os.environ["ACCELERATE_DISABLE_RICH"] = "1"
    from IPython import get_ipython
    ipython = get_ipython()
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-5e2524s2
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-5e2524s2
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 5afe6fed827592dd525490b81e213bc3e2241a4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for circuitsvis: filename=circuitsvis-0.0.0-py3-none-any.whl size=6170635 sha256=18416d046391a417a819352fdc50227fc53f6bd4b4a79fa57c3ef379f1518948
  Stored in directory: /tmp/pip-ephem-wheel-cache-6azqk56r/wheels/86/be/ad/78078aba9344d200aad61b63d35cdaecdec160212f039eed74
Successfully built circuitsvis
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.8.0
    Uninstalling importlib-metadata-6.8.0:
      Successfully uninstalled importlib-metadata-6.8.0


In [4]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [5]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import circuitsvis as cv
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [6]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [7]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    if isinstance(tensor, list):
        tensor = torch.stack(tensor)
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [8]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [9]:
def disable_biases(model):
    for name, param in model.named_parameters():
        if 'b_' in name:
            param.requires_grad = False

def disable_pos_embed(model):
    assert model.cfg.positional_embedding_type == "standard"
    model.pos_embed.W_pos = nn.Parameter(torch.zeros_like(model.pos_embed.W_pos))
    model.pos_embed.W_pos.requires_grad = False

# Load Model

The model is a 1L, 1H, attn-only transformer with no biases, layernorms, or positional embeddings. Note this is an extremely tiny model, yet was able to perform very well on this task. I've already trained the model, so below I just load it to this colab notebook:

In [10]:
torch.set_grad_enabled(False)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [12]:
MAX_ELT = 50
LIST_LEN = 10
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=64,
    attn_only=True,
    d_head=64,
    n_heads=1,
    normalization_type=None,
    d_vocab=MAX_ELT+2, # 0, ..., MAX_ELT-1, BOS, END
    n_ctx=LIST_LEN+2, #BOS a1 ... an END
    device=device,
    seed=0
)

model = HookedTransformer(cfg)

In [13]:
models_dir = "mech-interp-practice/models/"
filename = models_dir + "majority_element_model.pt"

state_dict = torch.load(filename)
model.load_state_dict(state_dict, strict=False);

# Task description

Recall the leetcode description: "Given an array nums of size n, return the majority element. The majority element is the element that appears more than $\lfloor n/2 \rfloor$ times. You may assume that the majority element always exists in the array”


The models inputs are always of the form:


[BOS, a1, a2, ..., a10, END]


Where BOS (51) and END (50) are special tokens at the start and end of every sequence. a1, ..., a10 are the array elements, which range between [0, 49] inclusive. Note that I fixed n=10 for every example during training. The model was trained to always output its prediction for the majority element at the END position.


Each example has a corresponding target, which is the majority element for that example. Below I provide the same data loader used in training, and some tokens / targets that you can use to start investigating the model:


In [14]:
def make_data_generator(cfg, batch_size, seed=0):
    torch.manual_seed(seed)
    random.seed(seed)
    BOS_TOKEN=cfg.d_vocab-1
    END_TOKEN=cfg.d_vocab-2
    bos_vec = torch.full((batch_size, 1), BOS_TOKEN)
    end_vec = torch.full((batch_size, 1), END_TOKEN)
    majority_num = LIST_LEN // 2 + 1
    while True:
        x = torch.randint(0, MAX_ELT, (batch_size, LIST_LEN))
        majority_elt = random.randint(0, MAX_ELT-1)
        majority_indices = torch.randperm(LIST_LEN)[:majority_num]
        x[:, majority_indices] = majority_elt
        yield torch.cat([bos_vec, x, end_vec], dim=-1), torch.full((batch_size,), majority_elt)

batch_size = 256
data_loader = make_data_generator(cfg, batch_size, seed=42)

tokens, targets = next(data_loader)
tokens, targets = tokens.to(device), targets.to(device)
print("tokens:", tokens.shape)
print(tokens[:5])
print("targets:", targets.shape)
print(targets[:5])

tokens: torch.Size([256, 12])
tensor([[51, 40, 40, 40, 40, 26, 40, 20, 40,  0, 13, 50],
        [51, 40, 40, 40, 40, 31, 40, 15, 40, 17,  6, 50],
        [51, 40, 40, 40, 40, 49, 40, 41, 40, 37, 19, 50],
        [51, 40, 40, 40, 40, 33, 40, 39, 40, 32, 10, 50],
        [51, 40, 40, 40, 40,  7, 40, 43, 40, 43, 27, 50]], device='cuda:0')
targets: torch.Size([256])
tensor([40, 40, 40, 40, 40], device='cuda:0')


I'll also provide the loss function used for training:

In [15]:
def loss_fn(logits, labels, per_token=False):
    if logits.ndim==3:
        logits=logits[:, -1, :]
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    if per_token:
        return -correct_log_probs
    else:
        return -correct_log_probs.mean()

It can be useful to convert tokens to a list of strings for plotly / CircuitsVis labels. Here is a function that you can optionally use for this:

In [16]:
def tokens_to_labels(tokens):
    if tokens.ndim == 2:
        tokens = tokens[0]
    res = [f"{tok}_{i}" for i, tok in enumerate(tokens)]
    res[0] = "BOS"
    res[-1] = "END"
    return res

print(tokens_to_labels(tokens))

['BOS', '40_1', '40_2', '40_3', '40_4', '26_5', '40_6', '20_7', '40_8', '0_9', '13_10', 'END']
